# 0.1.4: Build GBIF trait maps

The final step before training models using Earth observation (EO) data is to link the TRY trait data with the GBIF species observations and then grid them. In this way, we can have matching trait rasters to be paired with our EO data.

## Imports and config

In [1]:
from pathlib import Path

import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
import dask_geopandas as dgpd
import geopandas as gpd
import numpy as np
import pandas as pd
from src.conf.conf import get_config
from src.conf.environment import log

%load_ext autoreload
%autoreload 2

# Display all columns when printing a pandas DataFrame
pd.set_option("display.max_columns", None)

cfg = get_config()

cluster = LocalCluster(dashboard_address=":39143")
client = Client(cluster)

## Load GBIF and filter by PFT

Let's load the GBIF data and select only the "Tree" PFT.

In [2]:
def filter_pft(df: pd.DataFrame, pft_set: str, pft_col: str = "pft") -> pd.DataFrame:
    pfts = pft_set.split("_")
    if not any(pft in ["Shrub", "Tree", "Grass"] for pft in pfts):
        raise ValueError(f"Invalid PFT designation: {pft_set}")

    return df[df[pft_col].isin(pfts)]


gbif = (
    dd.read_parquet(Path(cfg.gbif.interim.dir, cfg.gbif.interim.subsampled))
    .pipe(filter_pft, "Tree")
    .repartition(npartitions=60)
    .sample(frac=0.01)
    .set_index("speciesname")
)

## Load TRY filtered mean trait data

In [3]:
mn_traits = (
    dd.read_parquet(Path(cfg.trydb.interim.dir, cfg.trydb.interim.filtered))
    .repartition(npartitions=60)
    .set_index("speciesname")
)

## Link mean trait values with GBIF data

In [4]:
merged = gbif.join(mn_traits, how="inner").reset_index()

In [14]:
# compute the number of unique indices
print(
    f"Pct matched species: {merged.index.nunique().compute() / gbif.index.nunique():.2%}"
)

Pct matched species: 61.89%


## Rasterize merged trait values

In [10]:
def global_grid_data(
    df: pd.DataFrame,
    traits: list[str],
    lon: str = "decimallongitude",
    lat: str = "decimallatitude",
    res: int | float = 0.5,
):
    """
    Modified from:
    https://sojwolf.github.io/iNaturalist_traits/Chapter_6_Compare_trait_maps_sPlot_iNat.html#grid-mean-trait-values-at-different-resolutions
    """
    # convert resolution in degrees into step size
    step = int((360 / res) + 1)

    bins_x = np.linspace(-180, 180, step)
    bins_y = np.linspace(-90, 90, int(((step - 1) / 2) + 1))

    # group latitude and longitude coordinates into bins
    df["x_bin"] = df.map_partitions(
        pd.cut, lon, bins=bins_x, meta=pd.Series(dtype="category", name="x_bin")
    )
    df["y_bin"] = df.map_partitions(
        pd.cut, lat, bins=bins_y, meta=pd.Series(dtype="category", name="y_bin")
    )

    df = df.dropna(subset=["x_bin", "y_bin"])
    
    # raster coordinates are in center of raster cell
    df["x_bin"] = df["x_bin"].map(lambda x: ((x.left + x.right) / 2))
    df["y_bin"] = df["y_bin"].map(lambda x: ((x.left + x.right) / 2))

    # group by 'x_bin' and 'y_bin' and calculate the mean of all variables
    grouped_df = df.groupby(["x_bin", "y_bin"])[traits].mean().reset_index()

    return grouped_df.compute()

In [11]:
cols = [col for col in merged.columns if col.startswith("X")]
grid_data = global_grid_data(merged.copy(), cols)

ValueError: Metadata inference failed in `map`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
AttributeError("'float' object has no attribute 'left'")

Traceback:
---------
  File "/home/dl1070/micromamba/envs/traits/lib/python3.12/site-packages/dask/dataframe/utils.py", line 195, in raise_on_meta_error
    yield
  File "/home/dl1070/micromamba/envs/traits/lib/python3.12/site-packages/dask_expr/_expr.py", line 3983, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/dl1070/micromamba/envs/traits/lib/python3.12/site-packages/dask/utils.py", line 1241, in __call__
    return getattr(__obj, self.method)(*args, **kwargs)
     

In [13]:
client.close()
cluster.close()